
#note reasoning for batches is in comparing_outliers.csv
#0 manual processing in interactive R session on cluster 
#1 to process each sample individually via slurm submission - avoiding as it results in different numbers of rows in count matrix of each sample which prevents merging later
#2 to process in bulk up to or including nnsvg step

skip ahead to 2


0. To manually run the preprocessing pipeline

In [ ]:

# #open interactive node session, in terminal connect to cluster then enter

srun --pty --job-name=interactive_r --mem=32G --cpus-per-task=4 --partition=epyc --time=024:00:00 bash 
#activate r environment with
micromambda activate r_env
#open R
R
Sys.setenv(PATH = paste(Sys.getenv("PATH"), "/home/lythgo02/micromamba/envs/r_env", sep = ":"))
#then run the commands in 20240204_QC_final_workflow.rmd but you will need to adapt the code for the samples you want to process as logbatch mitochondrial filters and the arbitrary mitochondrial filters batch

1. to run by as individual samples, open ssh terminal and set of individual slurm jobs for each (using a different script for the log batch and the arbitrary batch)

In [ ]:
#1.0

cd /mnt/scratchc/fmlab/lythgo02/visium_data/
for SAMPLE in SITSA3 SITSB2 SITSB4 SITSC1 SITSC3 SITSD3 SITSE4 SITSF2 SITSF4 ; 
do sbatch 
--job-name=${SAMPLE} 
--output=logs/${SAMPLE}.out 
--error=logs/${SAMPLE}.err 
--time=100:00:00 
--mem=32G 
--cpus-per-task=8 
--partition=epyc 
--wrap="Rscript single_sample_process_logmito.r ${SAMPLE}"; 
done

In [ ]:
#1.2

cd /mnt/scratchc/fmlab/lythgo02/visium_data/

for SAMPLE in SITSA1 SITSA2 SITSA4 SITSB1 SITSB3 SITSC2 SITSC4 SITSD1 SITSD2 SITSD4 SITSE2 SITSG2 SITSH2 ; 
 do sbatch 
 --job-name=${SAMPLE} 
 --output=logs/${SAMPLE}.out 
 --error=logs/${SAMPLE}.err 
 --time=048:00:00 
 --mem=32G 
 --cpus-per-task=8 
 --partition=epyc 
 --wrap="Rscript single_sample_processing_arbitrary.r ${SAMPLE}"; 
 done

once they have finished running cd into the respectiv folders and run the following

In [ ]:
#1.3

cd logbatch 

for SAMPLE in post_gettophgvSITSA3 post_gettophgvSITSB2 post_gettophgvSITSB4 post_gettophgvSITSC1 post_gettophgvSITSC3 post_gettophgvSITSD3 post_gettophgvSITSE4 post_gettophgvSITSF2 post_gettophgvSITSF4; 
 do     sbatch --job-name=${SAMPLE}            
 --output=logs/${SAMPLE}.out            
 --error=logs/${SAMPLE}.err            
 --time=0100:00:00            
 --mem=32G            
 --cpus-per-task=8            
 --partition=epyc            
 --wrap="Rscript /mnt/scratchc/fmlab/lythgo02/visium_data/logbatch/nnsvg_logbatch.r ${SAMPLE}"; 
 done

In [ ]:
#1.4

cd into arbitrary

for SAMPLE in post_gettophgvSITSA1 post_gettophgvSITSA2 post_gettophgvSITSA4 post_gettophgvSITSB1 post_gettophgvSITSB3 post_gettophgvSITSC2 post_gettophgvSITSC4 post_gettophgvSITSD1 post_gettophgvSITSD2 post_gettophgvSITSD4 post_gettophgvSITSE2 post_gettophgvSITSG2 post_gettophgvSITSH2;  
do     
sbatch 
--job-name=${SAMPLE}            
--output=logs/${SAMPLE}.out            
--error=logs/${SAMPLE}.err            
--time=0100:00:00            
--mem=32G            
--cpus-per-task=8            
--partition=epyc            
--wrap="Rscript /mnt/scratchc/fmlab/lythgo02/visium_data/arbitrary/nnsvg_arbitrary.r ${SAMPLE}";
done

#1.5 then pick up after running nnsvg 
#open interactive r session within micromamba r_rnv

In [ ]:
#1.5

knitr::opts_chunk$set(echo = TRUE)
library(spatialLIBD)
library(nnSVG)
library(SpatialExperiment)  
library(scater)
library(AnnotationHub)
library(tidyverse)
library(ggspavis)
library(scran)
library(DT)
projDir = "/mnt/scratchc/fmlab/lythgo02/visium_data/"


load_all_data <- "/mnt/scratchc/fmlab/lythgo02/visium_data/logbatch/nnsvg"
# List files in the directory
files <- list.files(path = load_all_data, pattern = "^nnsvg", full.names = TRUE)

# Print out the list of files
print(files)

#assign names so they are loaded into the spe with the correct identifiers
logbatch <- c("SITSA3", "SITSB2", "SITSB4", 
                 "SITSC1","SITSC3","SITSD3",
                 "SITSE4","SITSF2", "SITSF4")

names(files) <- logbatch
# Initialize a list to store each sample
spe_list <- list()

# Loop through each file and process it
for (sample_name in names(files)) {
  # Read the Visium data
  spe <- readRDS(files[sample_name])
    # Add unique barcodes
  spe$barcode <- rownames(colData(spe))
  spe$barcodeid <- gsub("-1$", paste0("-", sample_name), spe$barcode)
  rownames(colData(spe)) <- spe$barcodeid
  spe$sample_id <- sample_name

  colData(spe)$qc_mito <- colData(spe)$log_is_outlier_mad
  colData(spe) <- colData(spe)[ , !(colnames(colData(spe)) %in% c("log_mito_percent","log_median_mito_percent","log_mad_mito_percent","log_deviation_above_median","log_is_outlier_mad"))]

  # Store the processed sample in the list
  spe_list[[sample_name]] <- spe 
  
  
  assign(paste0("spe_", sample_name), spe)
  #print progress check
  cat("Loaded:", sample_name, "\n")
}

In [ ]:
#1.6 load arbitrary batch 

load_all_data <- "/mnt/scratchc/fmlab/lythgo02/visium_data/arbitrary/nnsvg"
# List files in the directory
files <- list.files(path = load_all_data, pattern = "^nnsvg", full.names = TRUE)

# Print out the list of files
print(files)
 
#assign names so they are loaded into the spe with the correct identifiers
arbitrary <- c("SITSA1","SITSA2","SITSA4", "SITSB1", "SITSB3", 
                 "SITSC2","SITSC4","SITSD1","SITSD2", "SITSD4",
                 "SITSE2","SITSG2", "SITSH2")

names(files) <- arbitrary 

# Loop through each file and process it
for (sample_name in names(files)) {
 # Read the Visium data
  spe <- readRDS(files[sample_name])
    # Add unique barcodes
  spe$barcode <- rownames(colData(spe))
  spe$barcodeid <- gsub("-1$", paste0("-", sample_name), spe$barcode)
  rownames(colData(spe)) <- spe$barcodeid
  spe$sample_id <- sample_name
  
  # Store the processed sample in the list
  spe_list[[sample_name]] <- spe 
  
  
  assign(paste0("spe_", sample_name), spe)
  #print progress check
  cat("Loaded:", sample_name, "\n")
}


In [ ]:
#1.6 bind all into one spatial object 


spe <- do.call(cbind, spe_list) #wont work because different row numbers # wont work if row numbers differ

2. To process all samples together in a slurm submission (either up to nnsvg or inc nnsvg)
#make sure you are updating and using the scratch scripts, there are also copies on mnt but outdated

In [ ]:
#2.0

sbatch /mnt/scratchc/fmlab/lythgo02/visium_data/scripts/nnsvg.sh

#which is 

#!/bin/bash

#SBATCH --error=nnSVG.err
#SBATCH --time=0200:00:00             
#SBATCH --cpus-per-task=32      
#SBATCH --mem=120G                   
#SBATCH --partition=epyc     


# Run the R script
#Rscript /mnt/scratchc/fmlab/lythgo02/visium_data/scripts/sample_specific_filtering_wholeworkflow.r 

In [ ]:
#there is also a script called end_stage_hvg_nnsvg_singlesamplejob.r which is the end section of the workflow to submit single jobs for each sample for hvg and nnsvg

In [ ]:
# 2.1 then process each sample separately for hvg and nnsvg via slurm job using following script 

# note after separating out individual samples, SITSA1 and SITSD3 have rows with 0 count (causing nnsvg to fail) for:
#SITSA1 - HNRNPCL3, TMOD4, REG1A USP17L10, PVRIG CNTNAP3B, ZP1, C13orf46 ZP2, PAGE2, TGIF2LY, PCDH11Y 
#SITSD3 - HNRNPCL3, PRAMEF33, UBE2U, CA14 AL590560.2, WDR64, SPDYA, NMS, ZDHHC19, USP17L10, HTN3, PCBD2, HIST1H4E, PVRIG 
         #ASZ1, AKR1C4, OOSP1, ZP1, DEFB131B, C13orf46, GOLGA6L6, TEPP, ALOX15, ANKRD30B, ZNF541, BPIFB2, BPIFB6, BPIFB3 
         #USP41, CRYBB2, GAGE1, PAGE2, SLC25A5, TGIF2LY, PRKY 
# remove zero expression genes
ix_zero_genes <- rowSums(counts(spe_SITSD3)) == 0
table(ix_zero_genes)

# identify genes with 0 expression
true <- ix_zero_genes[ix_zero_genes]

if (sum(ix_zero_genes) > 0) {
  spe_SITSD3 <- spe_SITSD3[!ix_zero_genes, ]}

# remove spots with zero expression
ix_zero_spots <- colSums(counts(spe_SITSA1)) == 0
table(ix_zero_spots) #0

In [ ]:
#2.1 
for sample in SITSA1 SITSA2 SITSA3 SITSA4 SITSB1 SITSB2 SITSB3 SITSB4 SITSC1 SITSC2 SITSC3 SITSC4 SITSD1 SITSD2 SITSD3 SITSD4 SITSE2 SITSE4 SITSF2 SITSF4 SITSG2 SITSH2; 
do     sbatch ../scripts/nnsvg.sh $sample; done

2.3 then open interactive R session and proceed as follows
We will take the top ranking HVGs and top ranking SVGs, reduce the dimensions with PCA on each, and then integrate and cluster


In [ ]:
srun --pty --job-name=interactive_r --mem=68G --cpus-per-task=4 --partition=epyc --time=100:00:00 bash

In [ ]:
#2.4 fitting top HVGs

knitr::opts_chunk$set(echo = TRUE)
library(spatialLIBD)
library(nnSVG)
library(SpatialExperiment)  
library(scater)
library(AnnotationHub)
library(tidyverse)
library(ggspavis)
library(scran)
library(DT)
projDir = "/mnt/scratchc/fmlab/lythgo02/visium_data/"



In [ ]:
#load samples
load_all_data <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/" 

# List files in the directory
files <- list.files(path = load_all_data, pattern = "spe_sub_modelgenevar", full.names = TRUE) #to load hvgs results
#hvg results stored in rowData(spe_ibject)$hvg

#assign names so they are loaded into the spe with the correct identifiers
samplesheet <- c("SITSA1", "SITSA2", "SITSA3", "SITSA4",
                 "SITSB1", "SITSB2", "SITSB3", "SITSB4", 
                 "SITSC1", "SITSC2", "SITSC3", "SITSC4", 
                 "SITSD1", "SITSD2", "SITSD3", "SITSD4",
                 "SITSE2", "SITSE4", "SITSF2", "SITSF4",
                 "SITSG2", "SITSH2")  

names(files) <- samplesheet

# Initialize a list to store each sample
spe_list <- list()

# Loop through each file and process it
for (sample_name in names(files)) {
 # Read the Visium data
  spe <- readRDS(files[sample_name])

  # Store the processed sample in the list
  spe_list[[sample_name]] <- spe 
  
  
  assign(paste0("spe_", sample_name), spe)
  #print progress check
  cat("Loaded:", sample_name, "\n")
}


Dimensionality reduction on top HVGs 

In [ ]:

#playing with pca and umap on hvg 

# Compute PCA for HVGs (highly variable genes)
set.seed(123)

#already run getTopHVGs and stored in hvg column of rowData(spe_object)$hvg
#pull out list of hvgs
#initialise list to store hvgs
hvg <- vector("list", length(spe_list))

for (i in seq_along(spe_list)) {
     #extract hvg list
     hvg[[i]] <- as.data.frame(rowData(spe_list[[i]])) %>%
     filter(hvg==TRUE) %>%
     pull(symbol)

 spe_list[[i]] <- runPCA(spe_list[[i]], subset_row = hvg[[i]])  
}
## Set seed
set.seed(987)
## Compute UMAP on top 50 PCs

for (i in seq_along(spe_list)) {

spe_list[[i]] <- runUMAP(spe_list[[i]], dimred = "PCA") 
}

for (i in seq_along(spe_list)) {

## Check correctness - names
print(reducedDimNames(spe_list[[i]])) 
#check correctness
print(dim(reducedDim(spe_list[[i]], "UMAP")))

## Update column names for easier plotting
colnames(reducedDim(spe_list[[i]], "UMAP")) <- paste0("UMAP", 1:2)
}

plots <- vector("list", length(spe_filt))
#plots
for (i in seq_along(spe_list)) {
     #extract sample id
     sample_id <- spe_list[[i]]$sample_id[i]
     
plots[[i]] <- ggplot(data = as.data.frame(spe_list[[i]]@int_colData@listData$reducedDims$PCA),
       aes(x = PC1, y = PC2, colour = spe_list[[i]]@colData$ground_truth)) + 
  geom_point(size = 0.5) + 
  scale_colour_brewer(type = "qual") + 
  labs(title = paste("Reduced dimensions: PCA (Sample:", sample_id, ")"),
       x = "PC1",
       y = "PC2",
       colour = "Layers") +
  theme_classic()
}

# Define the directory where you want to save the plots
save_dir <- "/mnt/scratchc/fmlab/lythgo02/visium_data/single_sample_from_filtering/"

for (i in seq_along(plots)) {
  # Define the file name based on the sample_id for uniqueness, with the full path
  file_name <- paste0(save_dir, "plot_", spe_list[[i]]$sample_id[1], ".png")
  
  # Save the plot using ggsave
  ggsave(filename = file_name, plot = plots[[i]], width = 8, height = 6)
}


2.6 Get top SVGs

nnsvg
The results are stored in the rowData of the SpatialExperiment object.
The main results of interest are:
LR_stat: likelihood ratio (LR) statistics used to rank SVGs
rank: rank of top SVGs according to LR statistics
pval: approximate p-values
padj: approximate p-values adjusted for multiple testing
prop_sv: effect size defined as proportion of spatial variance

The following section compares the most spatially variable genes for each sample to get the top consistently SVGs across all samples
Stored in df_summary

In [ ]:


#check if gene names are the same
# Initialize a list to store gene names for each sample
gene_names_list <- lapply(spe_list, function(spe) rownames(rowData(spe)))

# Compare each sample to the first one (assumed to be the reference)
reference_genes <- gene_names_list[[1]]
sample_names <- names(gene_names_list)

# Check if all gene names are in the same order
same_order <- sapply(gene_names_list, function(genes) identical(genes, reference_genes))

# Display results
if (all(same_order)) {
  cat("✅ All samples have gene names in the same order.\n")
} else {
  cat("❌ Some samples have gene names in a different order:\n")
  print(sample_names[!same_order])
}

#combining svgs
#rank svg results if gene names and order match in each object 

sample_ids <- names(spe_list)

# Get the number of genes (rows) from one of the spe objects, 
# assuming all spe objects have the same genes
num_genes <- nrow(rowData(spe_list[[1]]))

#matrix of svg ranks across samples 
# Initialize the res_ranks matrix with NA values
res_ranks <- matrix(NA, nrow = num_genes, ncol = length(spe_list))
rownames(res_ranks) <- rownames(rowData(spe_list[[1]]))  # Gene IDs from the row data of any spe object
colnames(res_ranks) <- names(spe_list)  # Sample names from the spe_list
# Loop through each sample in spe_list
for (s in seq_along(spe_list)) {
  sample_name <- names(spe_list)[s]  # Get the current sample name
  
  # Get the rank data from the current spe object
  gene_data <- as.data.frame(rowData(spe_list[[sample_name]]))
  
  # Check if the "rank" column exists
  if ("rank" %in% colnames(gene_data)) {
    # Extract ranks for the current sample
    ranks <- gene_data[, "rank", drop = FALSE]  # Retain ranks column as a data frame
    rownames(ranks) <- rownames(gene_data)  # Ensure gene names match
    
    # Assign the ranks to the appropriate column in res_ranks matrix
    res_ranks[rownames(ranks), sample_name] <- ranks$rank
  } else {
    stop("No 'rank' column found in the data for sample: ", sample_name)
  }
}

# remove genes that weren't ranked in any samples
ix_allna <- apply(res_ranks, 1, function(r) all(is.na(r)))
res_ranks <- res_ranks[!ix_allna, ]

dim(res_ranks)

# calculate average ranks
# note missing values due to filtering for samples
avg_ranks <- rowMeans(res_ranks, na.rm = TRUE)


# calculate number of samples where each gene is within top 100 ranked SVGs
# for that sample
n_withinTop100 <- apply(res_ranks, 1, function(r) sum(r <= 100, na.rm = TRUE))

df_summary <- data.frame(
  gene_id = rowData(spe_list[[1]])[names(avg_ranks), "ID"],  # Use "ID" for gene_id
  gene_name = rowData(spe_list[[1]])[names(avg_ranks), "symbol"],  # Use "symbol" for gene_name
  overall_rank = rank(avg_ranks),
  average_rank = unname(avg_ranks),
  n_withinTop100 = unname(n_withinTop100),
  row.names = names(avg_ranks)
)


# sort by average rank
df_summary <- df_summary[order(df_summary$average_rank), ]
head(df_summary)

In [ ]:
# number of significant SVGs
# #many are mitochondrial - should i filter out mitochrondial genes first or is it ok to filter out after ??
table(rowData(spe_filt)$padj <= 0.05)
## 

#show results for top 10 SVGs

test <- rowData(spe_filt)[order(rowData(spe_filt)$rank)[1:500], ] %>% 
  as.data.frame() #%>% 
  #filter(!str_detect(symbol,"MT-"))

# plot spatial expression of top-ranked SVG
ix <- which(rowData(spe_filt)$rank == 1)
ix_name <- rowData(spe_filt)$symbol[ix]
ix_name

In [ ]:
#plot spatial expression of a gene 

df <- as.data.frame(cbind(spatialCoords(spe_filt), expr = counts(spe_filt)[ix, ]))

df <- as.data.frame(cbind(
  spatialCoords(spe_filt), 
  expr = as.vector(as.matrix(counts(spe_filt))[ix, ])
))
ggplot(df, aes(x = pxl_col_in_fullres, y = pxl_row_in_fullres, 
               color = expr)) + 
  geom_point(size = 0.8) + 
  coord_fixed() + 
  scale_y_reverse() + 
  scale_color_gradient(low = "gray90", high = "blue", 
                       trans = "sqrt", breaks = range(df$expr), 
                       name = "counts") + 
  ggtitle(ix_name) + 
  theme_bw() + 
  theme(plot.title = element_text(face = "italic"), 
        panel.grid = element_blank(), 
        axis.title = element_blank(), 
        axis.text = element_blank(), 
        axis.ticks = element_blank())

#ggsave("20250227_nnsvg_sitsa2.png")

Define a function to extract top SVGs

In [ ]:
#extract top SVGs
#can simplify and just extract top n based on rank column 

getTopSVGs <- function(spe, top_n = 1000) {
  # Extract the row data from the spe object as a dataframe
  results <- as.data.frame(rowData(spe))
  
  # Sort by ascending rank
  results <- results[order(results$rank), ]

  # Select the top N SVGs (top 1000 by default)
  top_genes <- rownames(results)[seq_len(top_n)]

  # Create a logical vector for top SVGs
  svg_flag <- rownames(results) %in% top_genes

  # Add the 'svg' flag to rowData of the spe object
  rowData(spe)$svg <- svg_flag
  
  # Return the updated SpatialExperiment object
  return(spe)
}

# Now, apply the function to each SpatialExperiment object in spe_list
top_svg_list <- list()

for (i in seq_along(spe_list)) {
  spe <- spe_list[[i]]  # Extract the current SpatialExperiment object
  
  # Get top 1000 SVGs for this sample
  spe_updated <- getTopSVGs(spe, top_n = 1000)
  
  # Store the updated SpatialExperiment object in the list
  top_svg_list[[i]] <- spe_updated
}



In [ ]:
top_svgs <- getTopSVGs(spe_filt) #need to determine number to take ...same number as hvgs?

#extract symbols and convert to character list, also shorten to the same length as hvgs
top_svgs <- as.character(top_svgs[1:999, "symbol"])


Integrate SVGs and HVGs
A simple way to combine these features is to concatenate columns of principal components (PCs) calculated on the set of HVGs and the set of SVGs (excluding overlapping HVGs), and then using the combined set of features for further downstream analyses (Li et al. 2022).

Dimensionality reduction
Use PCA to reduce the dimensions of our dataset to assist clustering and UMAP to further reduce the principal components (PCs) in a two-dimensional space and produce better visualisations for the PCA.
runPCA() function runs PCA on a SCE object, and returns an updated version of the single cell object with the PCA result added to the reducedDim slot.

In [ ]:
# Compute PCA for HVGs (highly variable genes)
set.seed(123)
spe_filt <- runPCA(spe_filt, subset_row = top_hvgs)  # Restrict PCA to use only HVGs
# Store the PCA result for HVGs in the "PCA_HVGs" slot
#After running runPCA(), the result is automatically stored in a default reduced dimension slot named "PCA". Assign to PCA_HVGs so it wont be overwritten by svg pca
reducedDim(spe_filt, "PCA_HVGs") <- reducedDim(spe_filt, "PCA")
# Optional: Check the results
reducedDimNames(spe_filt)

# Remove the PCA results for HVGs (if you want to overwrite or free up memory)
#reducedDim(spe_filt, "PCA") <- NULL


# Compute PCA for SVGs (spatially variable genes)
set.seed(123)
spe_filt <- runPCA(spe_filt, subset_row = top_svgs)  # Restrict PCA to use only SVGs

reducedDim(spe_filt, "PCA_SVGs") <- reducedDim(spe_filt, "PCA")
# Optional: Check the results
reducedDimNames(spe_filt)

#CHECK HVGS AND SVGS IN PCA SLOTS ARE DIFFERENT

In [ ]:

# View the dimensions of the reduced dimensions for both PCA analyses
dim(reducedDim(spe_filt, "PCA_HVGs"))
dim(reducedDim(spe_filt, "PCA_SVGs"))

# View the variance explained for both PCA analyses
percent.var.hvgs <- attr(reducedDim(spe_filt, "PCA_HVGs"), "percentVar")
percent.var.svgs <- attr(reducedDim(spe_filt, "PCA_SVGs"), "percentVar")

# Plot variance explained for HVGs
# Specify the file path and name for the plot (e.g., "plot.png")
png("variance_explained_hvgs_pca_sitsa2.png", width = 800, height = 600)

plot(percent.var.hvgs, log = "y", xlab = "PC", ylab = "Variance explained (%)")
abline(v = PCAtools::findElbowPoint(percent.var.hvgs), col = "dodgerblue")

# Close the PNG device to save the plot
dev.off()

# Specify the file path and name for the plot (e.g., "plot.png")
png("variance_explained_svgs_pca_sitsa2.png", width = 800, height = 600)

# Plot variance explained for SVGs
plot(percent.var.svgs, log = "y", xlab = "PC", ylab = "Variance explained (%)")
abline(v = PCAtools::findElbowPoint(percent.var.svgs), col = "dodgerblue")

# Close the PNG device to save the plot
dev.off()

ONLY GOT AS FAR AS RUNNING TO HERE, CONTINUE 

In [ ]:
#Integrate HVGs and SVGs for clustering 
# Extract the PCs for both HVGs and SVGs
pca_hvgs <- reducedDim(spe_filt, "PCA_HVGs")
pca_svgs <- reducedDim(spe_filt, "PCA_SVGs")

# Concatenate the principal components (PCs)
combined_pcs <- cbind(pca_hvgs, pca_svgs)

# Store the concatenated PCs in the 'spe_filt' object
reducedDim(spe_filt, "PCA_combined") <- combined_pcs

# Check the result
dim(reducedDim(spe_filt, "PCA_combined"))

In [ ]:
#Run UMAP on the set of top 50 PCs and retain top 2 UMAP components for visualization 
# compute UMAP on top 50 PCs
set.seed(123)
spe_filt <- runUMAP(spe_filt, dimred = "PCA")

reducedDimNames(spe_filt)
dim(reducedDim(spe_filt, "UMAP"))

# update column names for easier plotting
colnames(reducedDim(spe_filt, "UMAP")) <- paste0("UMAP", 1:2)

# plot top 2 PCA dimensions
plotDimRed(spe_filt, type = "PCA")

# plot top 2 UMAP dimensions
plotDimRed(spe_filt, type = "UMAP")
```

In [ ]:
sessionInfo()